In [1]:
import numpy as np
import scipy 
import scipy.linalg
import matplotlib.pyplot as plt
from scipy.integrate import quad
from tqdm import tqdm,trange

|  | min   | max  | average  | stdev  |
|---------------------|---|---|---|---|
| $\sigma_{i,\infty}^2$ | 0.00060%  | 0.05150% | 0.01770%  | 0.01440% |
| $\alpha_i$  | 0.0329  | 0.0955 | 0.0622 | 0.0242 |
| $\beta_i$  | 0.8855  | 0.9656  | 0.9306 | 0.0271 |
| $\omega_i$  | 0.0000032% | 0.0004320% | 0.0001350%| 0.0001370% |
| $\rho_{i, j}$  | -22%  | 82%  | 10& | 22% |
| $p_{max,i}$  | 1.83  |5.72 | 3.34 | 1.22 |
| $t_{1/2,i}$  | 36.21 | 437.17 | 135.78 | 98.52 |

$$r_t=D_t\Gamma^{1/2}\eta_t=D_t\tilde{\eta_t},$$ $$V_t=D_t\Gamma D_t,$$
$$\sigma_{i,t}^2 = \omega_{i,t}+\alpha_ir_{i,t-1}^2+\beta_i\alpha_{i,t-1}^2=(1, r_{i,t-1}^2, \sigma_{i,t-1}^2)\theta_i, \, \theta_i=(\omega_i, \alpha_i, \beta_i)^T$$

### Robins Monro with $\theta$ constant

In [2]:
Theta = np.array([[1.35e-4, 3.2e-6, 4.32e-4], [0.0622, 0.0329, 0.0955], [0.9306, 0.8855, 0.9656]])
d = 3

In [3]:
def parameters_simulator(r_pre, sigma_pre, GAMMA, T):
    """
    simulate (theta, r_{T+1})
    Input:
        
        r_pre: ndarray, return initialization
        sigma_pre: ndarray, components for the diagonal matrix D
        GAMMA: (n, n) array for constant correlation matrix
        T: Time for the process
    """

    D_t_pre = np.diag(sigma_pre)
    R_t_pre = np.diag(r_pre)
    
    for t in range(T):
        
        theta = np.zeros((n, 3))
        for j in range(3):
            theta[: j] = np.random.uniform(Theta[j][1], Theta[j][2])
            
        D_t_2 = np.eye(n)*theta[:,0] + theta[:,1]*R_t_pre**2 + theta[:,2]*D_t_pre**2
        D_t = np.sqrt(D_t_2)
        V_t = D_t.dot(GAMMA).dot(D_t)
        r_t = np.random.multivariate_normal(np.zeros(n), V_t)
        D_t_pre, R_t_pre = D_t, np.diag(r_t)
        
    return theta, r_t

In [4]:
def r_simulator(theta, GAMMA, T):
    """
    simulate  r_{T+1}
    Input:
        theta: n*3 array, (omega, alpha, beta)
        GAMMA: (n, n) array for constant correlation matrix
        T: Time for the process (should be larger than t_0.5 = -ln2/ln(alpha+beta))
    """
    r_init = theta[:, 0]/(1-theta[:, 1]-theta[:, 2])
    D_t_pre = np.diag(np.random.uniform(6e-4, 5.15e-2, n))
    R_t_pre = np.diag(r_int)
    
    for t in range(T):
        D_t_2 = np.eye(n)*theta[:,0] + theta[:,1]*R_t_pre**2 + theta[:,2]*D_t_pre**2
        D_t = np.sqrt(D_t_2)
        V_t = D_t.dot(GAMMA).dot(D_t)
        r_t = np.random.multivariate_normal(np.zeros(n), V_t)
        D_t_pre, R_t_pre = D_t, np.diag(r_t)
        
    return r_t
    


In [5]:
def parameters_simulator(GAMMA, N_samples):
    """
    simulate N samples of (theta, r_{T+1})
    """
    Theta = []
    R = []
    for i in range(N_samples):
        theta = np.zeros((n, 3))
        for j in range(3):
            theta[: j] = np.random.uniform(Theta[j][1], Theta[j][2])
        T = int((-5* np.log(2)/np.log(theta[:, 1]+theta[:, 2])).max())
        
        r = r_simulator(theta, GAMMA, T)
        
        Theta.append(theta)
        R.append(r)
    return Theta, R


In [6]:
def R_simulator(R0, sigma2_0, theta, Gamma):
    ones = np.ones((d,1))
    g = np.hstack((ones, R0**2, sigma2_0))
    sigma2 = g.dot(theta)
    D = np.diag(np.sqrt(sigma2))
    eta = np.sqrt(Gamma).dot(np.random.randn(d))  
    return D.dot(eta)

In [7]:
np.hstack((np.ones((d,1)),np.ones((d,1))))

array([[1., 1.],
       [1., 1.],
       [1., 1.]])

In [8]:
from scipy.stats import random_correlation

In [9]:
np.random.seed(123)
corr = np.diag(np.ones(d))
for i in range(d):
    for j in range(i+1, d):
        corr[i, j] = corr[j, i] = np.random.uniform(-0.2, 0.8)
corr

array([[1.        , 0.49646919, 0.08613933],
       [0.49646919, 1.        , 0.02685145],
       [0.08613933, 0.02685145, 1.        ]])

In [10]:
import scipy.linalg
e, v = scipy.linalg.eigh(corr)
e

array([0.49997036, 0.99097736, 1.50905228])

In [11]:
def H(x, r, theta):
    """
    The function for the iteration
    Input:
        x: (n-1, 1) ndarray
        r: (n, 1) ndarray
        theta: (d, 3) ndarray
    """
    max_norm = np.linalg.norm(theta, axis = 1).max()
    left = A.T.dot(r.dot(t.T))
    right = A.dot(x) + b
    return left.dot(right)/max_norm


def iteration(self):
    u0 = np.array(self.u0)
    u = u0
    for k in range(1, self.K+1):

        theta = []
        r_ts_set = []
        for _ in range(self.M[k]+1):
            the,rts = self.para_simulate()
            theta.append(the)
            r_ts_set.append(rts)

        keyboard = self.keboard
        for key in keyboard[self.m[k] + 1]:
            expect = 0.0
            x = np.zeros(n-1)
            for k in keyborad[m[k-1]]:
                x += u[k] * self.LB[k](theta)
            for s in range(1,self.M[k]+1):
                expect += (self.LB[key](theta[s]) * self.H(x, r_ts_set[s])).reshape(1,n)
            expect /= self.M[k]

            u[key] = u.get(key, np.zeros(n-1)) - self.gamma[k] * expect

        u0 = u
        self.u_record.append(u)
    return u

In [53]:
def hyperbolic_cross_index_set(d, deg):
    """
    compute the hyperbolic index set in d-dimensional case
    deg: Degree, a nonnegative integer
    
    Output:
        a list consisting of all the hyperbolic index
    """
    
    if deg == 0:
        return [[0]*d]
    
    index_set = hyperbolic_cross_index_set(d, deg - 1)
    
    for s in index_set: 
        for i in range(d):
            s_ = s.copy()
            s_[i] = s[i]+1
            tmp = [max(p,q) for p, q in zip(s_, np.ones(d))]
            if s_ not in index_set and s_[i] <= deg and np.prod(tmp)<=deg:
                index_set.append(s_)
    return index_set
             
hyperbolic_cross_index_set(3,3)

[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [1, 1, 0],
 [1, 0, 1],
 [0, 1, 1],
 [1, 1, 1],
 [2, 0, 0],
 [0, 2, 0],
 [0, 0, 2],
 [2, 1, 0],
 [1, 2, 0],
 [2, 0, 1],
 [1, 0, 2],
 [0, 2, 1],
 [0, 1, 2],
 [2, 1, 1],
 [1, 2, 1],
 [1, 1, 2],
 [3, 0, 0],
 [0, 3, 0],
 [0, 0, 3],
 [3, 1, 0],
 [1, 3, 0],
 [3, 0, 1],
 [1, 0, 3],
 [0, 3, 1],
 [0, 1, 3],
 [3, 1, 1],
 [1, 3, 1],
 [1, 1, 3]]